In [1]:
import logging
from pathlib import Path
from mne.time_frequency.tfr import tfr_multitaper
from mne_bids.path import get_entities_from_fname
import numpy as np
import pandas as pd
import mne
from mne_bids import BIDSPath, get_entity_vals
from mne.time_frequency import stft
from mne import make_fixed_length_epochs

from eztrack.utils import logger

import sys
import os

sys.path.append("../")
sys.path.append("./episcalp")
print(os.getcwd())
from episcalp.io import read_scalp_eeg

logger.setLevel(logging.DEBUG)


/Users/adam2392/Documents/episcalp/notebooks


In [13]:
root = Path("/Users/adam2392/Johns Hopkins/Scalp EEG JHH - Documents/bids/")
deriv_root = root / "derivatives"
bids_root = deriv_root / 'ICA' / '1-30Hz-30' / 'win-20'

figures_path = deriv_root

# define BIDS entities
IGNORE_SUBJECTS = []
FREQ_BANDS = {
    'delta': np.arange(1, 4, 0.5),
    'theta': np.arange(4, 8, 0.5),
    'alpha': np.arange(8, 12, 0.5),
    'beta': np.arange(12, 30, 1)
}
freq_band = 'delta'

deriv_chain = Path('tfr') / freq_band

datatype = "eeg"
extension = ".edf"
n_jobs = -1
# analysis parameters
reference = "monopolar"
sfreq = None
overwrite = False

# get the runs for this subject
all_subjects = get_entity_vals(bids_root, "subject")

for subject in all_subjects:
    if subject in IGNORE_SUBJECTS:
        continue

    ignore_subs = [sub for sub in all_subjects if sub != subject]
    subj_dir = bids_root / f"sub-{subject}"

    fpaths = list(subj_dir.rglob('*.edf'))

    print(f"Found filepaths for {subject}: {fpaths}.")

    for idx, fpath in enumerate(fpaths):
        entities = get_entities_from_fname(fpath.name)

        # create path for the dataset
        bids_path = BIDSPath(
            **entities,
            datatype=datatype,
            root=bids_root,
            extension=extension,
        )
        print(f"Analyzing {bids_path}")

        raw = read_scalp_eeg(bids_path, reference=reference)

        # run TFR analysis
        epochs = make_fixed_length_epochs(raw, duration=2, overlap=1)

        freqs = FREQ_BANDS[freq_band]

        epochs_tfr  = tfr_multitaper(epochs, freqs, n_jobs=n_jobs, n_cycles = freqs/2., return_itc=False, average=False)
        
        deriv_fpath = deriv_root / deriv_chain / f'sub-{subject}' / bids_path.copy().update(
            check=False, suffix=f'desc-{freq_band}_eeg-tfr.h5').basename
        deriv_fpath.parent.mkdir(exist_ok=True, parents=True)
        print(f'Saving to {deriv_fpath}')
        epochs_tfr.save(deriv_fpath)
        break
        
    break

Found filepaths for jhh001: [PosixPath('/Users/adam2392/Johns Hopkins/Scalp EEG JHH - Documents/bids/derivatives/ICA/1-30Hz-30/win-20/sub-jhh001/eeg/sub-jhh001_run-01_eeg.edf')].
Analyzing /Users/adam2392/Johns Hopkins/Scalp EEG JHH - Documents/bids/derivatives/ICA/1-30Hz-30/win-20/sub-jhh001/eeg/sub-jhh001_run-01_eeg.edf
Looked at ['standard_1020', 'EGI_256', 'GSN-HydroCel-128', 'GSN-HydroCel-129', 'GSN-HydroCel-256', 'GSN-HydroCel-257', 'GSN-HydroCel-32', 'GSN-HydroCel-64_1.0', 'GSN-HydroCel-65_1.0', 'biosemi128', 'biosemi16', 'biosemi160', 'biosemi256', 'biosemi32', 'biosemi64', 'easycap-M1', 'easycap-M10', 'mgh60', 'mgh70', 'standard_1005', 'standard_1020', 'standard_alphabetic', 'standard_postfixed', 'standard_prefixed', 'standard_primed', 'artinis-octamon', 'artinis-brite23'] to find good match...
Going to use standard_1020 as the montage.
Going to drop [] channels...
Dropping these channels that are not in standard_1020 montage:  []


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  13 out of  16 | elapsed:    1.1s remaining:    0.3s
[Parallel(n_jobs=8)]: Done  16 out of  16 | elapsed:    1.2s finished


Saving to /Users/adam2392/Johns Hopkins/Scalp EEG JHH - Documents/bids/derivatives/tfr/delta/sub-jhh001/sub-jhh001_run-01_desc-delta_eeg-tfr.h5.edf


In [7]:
print(epochs)
print(epochs.events)

<Epochs |  653 events (all good), 0 - 1.995 sec, baseline off, ~33 kB, data not loaded,
 '1': 653>
[[     0      0      1]
 [   300      0      1]
 [   600      0      1]
 ...
 [195000      0      1]
 [195300      0      1]
 [195600      0      1]]


In [19]:
print(epochs_tfr.info)
print(epochs_tfr)
print(epochs_tfr.events)
print(epochs_tfr.event_id)

<Info | 10 non-empty values
 bads: []
 ch_names: Fp1, Fp2, F3, F4, C3, C4, P3, P4, O1, O2, F7, F8, T3, T4, T5, T6
 chs: 16 EEG
 custom_ref_applied: False
 dig: 22 items (3 Cardinal, 19 EEG)
 highpass: 1.0 Hz
 line_freq: 60
 lowpass: 30.0 Hz
 meas_date: 2021-09-28 10:30:19 UTC
 nchan: 16
 projs: []
 sfreq: 200.0 Hz
 subject_info: 6 items (dict)
>
<EpochsTFR | time : [0.000000, 1.995000], freq : [1.000000, 3.500000], epochs : 980, channels : 16, ~287.1 MB>
[[     0      0      1]
 [   200      0      1]
 [   400      0      1]
 ...
 [195400      0      1]
 [195600      0      1]
 [195800      0      1]]
{'1': 1}
